In [1]:
#import all neccessary libraries
import pandas as pd
import numpy as np
import re
import string
import nltk
import sklearn
nltk.download("all")
nltk.download('wordnet')
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder
import textblob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('max_colwidth', 100)
import warnings
warnings.filterwarnings('ignore')



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [2]:
#import dataframe
data=pd.read_csv("/content/sample.csv")

In [3]:
#check data
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,"@105835 Your business means a lot to us. Please DM your name, zip code and additional details ab...",NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm sure you won't! Because you don't have to!,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 opti...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've tried leaving a voicemail several times in the pa...,119243,119240.0


In [4]:
data["text"][0]

'@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡'

In [5]:
data["text"]

0     @AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard...
1     @105835 Your business means a lot to us. Please DM your name, zip code and additional details ab...
2                  @76328 I really hope you all change but I'm sure you won't! Because you don't have to!
3     @105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 opti...
4     @VirginTrains see attached error message. I've tried leaving a voicemail several times in the pa...
                                                     ...                                                 
88    @105860 I wish Amazon had an option of where I can just get it shipped to the ups store so I can...
89                                           They reschedule my shit for tomorrow https://t.co/RsvZcT982t
90    @105861 Hey Sara, sorry to hear of the issues you are having, can I ask if it's the lay out or j...
91    @Tesco bit of both - finding the layout 

In [6]:
data["text"][10]

'@105837 We can help. Which version of iOS are you on? You can find that in Settings &gt; General &gt; About. Reply in DM. https://t.co/GDrqU22YpT'

In [7]:

# Function to replace multiple substrings
def replace_multiple(string, replacements):
    new_string = string
    for old_value, new_value in replacements.items():
        new_string = new_string.replace(old_value, new_value)
    return new_string

# Dictionary of replacements
replacements = {"&gt;": ">", "&lt;": "<", "&amp;": "&"}

# Apply the function to the DataFrame column
data['text'] = data['text'].apply(lambda x: replace_multiple(x, replacements))



In [8]:
#remove url function
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("",text)


data["text"]=data["text"].apply(remove_url)

In [9]:
# Function to remove hashtags
def remove_hashtags(sentence):
    words = sentence.split()
    without_hashtags = [word for word in words if not word.startswith('#')]
    return ' '.join(without_hashtags)

# Apply the function to the DataFrame column
data['text'] = data['text'].apply(lambda x: remove_hashtags(x))



In [10]:
data["text"][24]

'@AppleSupport after the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently'

In [11]:
#remove punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
#collect in variable exclude
exclude=string.punctuation

In [13]:
#remove punctuation
def remove_punc(text):
    return text.translate(str.maketrans("","",exclude))

data["text"]=data["text"].apply(remove_punc)

In [14]:
data["text"][10]

'105837 We can help Which version of iOS are you on You can find that in Settings  General  About Reply in DM'

In [15]:
#remove Slang
chat_words={
   "DM":"Direct Message",
   "TY":"Thank You",
   "AC":"Account",
"AY": "Assessment Year",
"HP":"Happy",
"CP":"coupling",
"MM":"Message me",
"AD":"Advertisement",
"MU":"Miss You",
"RR":"Reply Requested",
   "JR":"Junior"

}

In [16]:
#remove slang and apply function
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [17]:
#apply function
data["text"]=data["text"].apply(chat_conversion)

In [18]:
data["text"][10]

'105837 We can help Which version of iOS are you on You can find that in Settings General About Reply in Direct Message'

In [19]:
data["text"][9:45]

9                105836 Thats what were here for Miriam 😊 The team should send you an email shortly Happy
10    105837 We can help Which version of iOS are you on You can find that in Settings General About R...
11                                   105838 AppleSupport Me too am suffering hope the can find a solution
12    AppleSupport hi I’ve a concern about the latest ios is too slow on and i am not happy with it An...
13    105839 Thanks for reaching out to us We are always happy to help Send us a Direct Message so we ...
14    I just updated my phone and suddenly everything takes ages to load wtf 76099 this update sux I h...
15    105840 Hi there What device is this happening on If you could also let us know the Android and S...
16    SpotifyCares Thanks Version 8422857 armv7 on anker bluetooth speaker on Samsung Galaxy Tab A 201...
17    105840 Thanks The distance could possibly affect playback Does logging out restarting your devic...
18    SpotifyCares No but Ive moved speaker to

In [20]:
#remove Emoji
def remove_emojis_manually(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text

In [21]:
data["text"]=data["text"].apply(remove_emojis_manually)

In [22]:
#remove numbers from sentance
def remove_numbers(sentence):

    return re.sub(r'\d+', '', sentence)

# Apply the function to the DataFrame column
data["text"] = data["text"].apply(remove_numbers)

In [23]:
data["text"][0]

'AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened'

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
print(data["text"])

0     AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard ...
1      Your business means a lot to us Please Direct Message your name zip code and additional details...
2                              I really hope you all change but Im sure you wont Because you dont have to
3         LiveChat is online at the moment or contact    option    Leave a message to request a call back
4     VirginTrains see attached error message Ive tried leaving a voicemail several times in the past ...
                                                     ...                                                 
88     I wish Amazon had an option of where I can just get it shipped to the ups store so I can avoid ...
89                                                                   They reschedule my shit for tomorrow
90     Hey Sara sorry to hear of the issues you are having can I ask if its the lay out or just the sp...
91    Tesco bit of both finding the layout cum

In [28]:


stemmer = PorterStemmer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    # stem
    #words = [stemmer.stem(word) for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document



In [29]:
# extract the messages from the dataframe
messages = [message for message in data["text"]]
print(messages)

['AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened', ' Your business means a lot to us Please Direct Message your name zip code and additional details about your concern Reply Requested', ' I really hope you all change but Im sure you wont Because you dont have to', ' LiveChat is online at the moment or contact    option    Leave a message to request a call back', 'VirginTrains see attached error message Ive tried leaving a voicemail several times in the past week', ' Have you tried from another device Miriam Message me', 'VirginTrains yep Ive tried laptop too several times over the past week and again today Ive tried different browsers too', ' Its working OK from here Miriam Does this link help Message me', 'VirginTrains I still havent heard the number Im directed to by phone is a dead end the live chat doesnt work Can someone call me', ' Thats what were here for Miriam  The team should send you an email shortly Happy', ' We can

In [30]:
documents = [preprocess(document) for document in messages]
print(documents)

['applesupport causing reply disregarded tapped notification keyboard opened', 'business means lot us please direct message name zip code additional details concern reply requested', 'really hope change im sure wont dont', 'livechat online moment contact option leave message request call back', 'virgintrains see attached error message ive tried leaving voicemail several times past week', 'tried another device miriam message', 'virgintrains yep ive tried laptop several times past week today ive tried different browsers', 'working ok miriam link help message', 'virgintrains still havent heard number im directed phone dead end live chat doesnt work someone call', 'thats miriam team send email shortly happy', 'help version ios find settings general reply direct message', 'applesupport suffering hope find solution', 'applesupport hi ’ concern latest ios slow happy solution please', 'thanks reaching us always happy help send us direct message look together', 'updated phone suddenly everythin

In [31]:
#Creating bag of words model using count vectorizer function

vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(documents)
print(bow_model)

  (0, 27)	1
  (0, 72)	1
  (0, 356)	1
  (0, 125)	1
  (0, 434)	1
  (0, 294)	1
  (0, 229)	1
  (0, 304)	1
  (1, 356)	1
  (1, 58)	1
  (1, 269)	1
  (1, 260)	1
  (1, 473)	1
  (1, 323)	1
  (1, 121)	1
  (1, 270)	1
  (1, 288)	1
  (1, 509)	1
  (1, 84)	1
  (1, 1)	1
  (1, 117)	1
  (1, 91)	1
  (1, 358)	1
  (2, 344)	1
  (2, 197)	1
  :	:
  (91, 101)	1
  (91, 354)	1
  (91, 219)	1
  (91, 146)	1
  (91, 202)	1
  (91, 393)	1
  (91, 227)	1
  (91, 448)	1
  (91, 176)	1
  (91, 247)	1
  (92, 323)	1
  (92, 121)	1
  (92, 270)	1
  (92, 288)	1
  (92, 118)	1
  (92, 189)	1
  (92, 127)	1
  (92, 134)	1
  (92, 442)	1
  (92, 163)	1
  (92, 2)	1
  (92, 209)	1
  (92, 55)	1
  (92, 254)	1
  (92, 242)	1


In [32]:
#vector vocabulary
vectorizer.vocabulary_

{'applesupport': 27,
 'causing': 72,
 'reply': 356,
 'disregarded': 125,
 'tapped': 434,
 'notification': 294,
 'keyboard': 229,
 'opened': 304,
 'business': 58,
 'means': 269,
 'lot': 260,
 'us': 473,
 'please': 323,
 'direct': 121,
 'message': 270,
 'name': 288,
 'zip': 509,
 'code': 84,
 'additional': 1,
 'details': 117,
 'concern': 91,
 'requested': 358,
 'really': 344,
 'hope': 197,
 'change': 74,
 'im': 206,
 'sure': 425,
 'wont': 498,
 'dont': 129,
 'livechat': 251,
 'online': 302,
 'moment': 283,
 'contact': 94,
 'option': 307,
 'leave': 240,
 'request': 357,
 'call': 64,
 'back': 41,
 'virgintrains': 479,
 'see': 370,
 'attached': 37,
 'error': 138,
 'ive': 220,
 'tried': 459,
 'leaving': 241,
 'voicemail': 481,
 'several': 378,
 'times': 453,
 'past': 316,
 'week': 487,
 'another': 22,
 'device': 118,
 'miriam': 278,
 'yep': 505,
 'laptop': 234,
 'today': 454,
 'different': 120,
 'browsers': 56,
 'working': 500,
 'ok': 298,
 'link': 248,
 'help': 189,
 'still': 415,
 'havent'

In [33]:
# print the full sparse matrix
print(bow_model.toarray())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [34]:
print(bow_model.shape)
print(vectorizer.get_feature_names_out())

(93, 510)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ages' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow'
 'almost' 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another'
 'answer' 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps'
 'armv' 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance'
 'attached' 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg'
 'behaviors' 'believes' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairways' 'broadside' 'broken' 'browser' 'browsers' 'bug'
 'business' 'buying' 'bye' 'cable' 'cache' 'cachecookies' 'call' 'called'
 'calling' 'calls' 'cares' 'carry' 'cause' 'caused' 'causing' 'challenged'
 'change' 'charge' 'chat' 'check' 'cheers' 'choice' 'chris' 'click'
 'close' 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'comes'
 'commonsense' 'companion' 'concern' 'congrats' 'constantly' 'contact'
 'cookies' 'correct' 'could' 'coupling' 'crash' 'crossed' 'cumbersome'
 'curiousw

Stemming and lemmatising

In [35]:


stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document, stem=True):
    'changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

Bag of words model on stemmed messages

In [36]:
# stem messages
stem_messages = [preprocess(message, stem=True) for message in  documents]

# bag of words model
vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(stem_messages)

In [37]:
# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,addit,address,adnam,advertis,advis,affect,age,ago,agoura,...,work,wtf,xfiniti,yard,year,yep,youll,your,youv,zip
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
stem_messages

['applesupport caus repli disregard tap notif keyboard open',
 'busi mean lot us pleas direct messag name zip code addit detail concern repli request',
 'realli hope chang im sure wont dont',
 'livechat onlin moment contact option leav messag request call back',
 'virgintrain see attach error messag ive tri leav voicemail sever time past week',
 'tri anoth devic miriam messag',
 'virgintrain yep ive tri laptop sever time past week today ive tri differ browser',
 'work ok miriam link help messag',
 'virgintrain still havent heard number im direct phone dead end live chat doesnt work someon call',
 'that miriam team send email shortli happi',
 'help version io find set gener repli direct messag',
 'applesupport suffer hope find solut',
 'applesupport hi ’ concern latest io slow happi solut pleas',
 'thank reach us alway happi help send us direct messag look togeth',
 'updat phone suddenli everyth take age load wtf updat sux hate fix bye',
 'hi devic happen could also let us know android 

In [39]:
print(vectorizer.get_feature_names_out())

['account' 'addit' 'address' 'adnam' 'advertis' 'advis' 'affect' 'age'
 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost' 'also' 'alway'
 'amanda' 'amazon' 'android' 'anker' 'anoth' 'answer' 'anyth' 'app' 'appl'
 'applesupport' 'appreci' 'armv' 'articl' 'ask' 'askspectrum' 'ass'
 'assess' 'assist' 'attach' 'avoid' 'away' 'ba' 'back' 'basket' 'batteri'
 'beatner' 'beg' 'behavior' 'believ' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairway' 'broadsid' 'broken' 'browser' 'bug' 'busi' 'buy' 'bye'
 'cabl' 'cach' 'cachecooki' 'call' 'care' 'carri' 'caus' 'challeng'
 'chang' 'charg' 'chat' 'check' 'cheer' 'choic' 'chri' 'click' 'close'
 'coaxial' 'code' 'colleagu' 'come' 'commonsens' 'companion' 'concern'
 'congrat' 'constantli' 'contact' 'cooki' 'correct' 'could' 'coupl'
 'crash' 'cross' 'cumbersom' 'curiouswil' 'current' 'custom' 'dani' 'day'
 'dead' 'delay' 'delet' 'deliv' 'deliveri' 'depart' 'departur' 'detail'
 'devic' 'differ' 'direct' 'disconnect' 'disgrac' 'disregard' 'dis

Now Apply lemmatizing the messages.

In [40]:
# lemmatise messages
lemma_messages = [preprocess(message, stem=False) for message in documents]

# bag of words model
vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(lemma_messages)

In [41]:
# look at the dataframe
pd.DataFrame(bow_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ago,agoura,...,work,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


After applying stemming and Lemmatization  got 453 and 459 tokens out of 510 tokens

In [42]:
lemma_messages

['applesupport cause reply disregard tap notification keyboard open',
 'business mean lot us please direct message name zip code additional detail concern reply request',
 'really hope change im sure wont dont',
 'livechat online moment contact option leave message request call back',
 'virgintrains see attach error message ive try leave voicemail several time past week',
 'try another device miriam message',
 'virgintrains yep ive try laptop several time past week today ive try different browsers',
 'work ok miriam link help message',
 'virgintrains still havent hear number im direct phone dead end live chat doesnt work someone call',
 'thats miriam team send email shortly happy',
 'help version ios find settings general reply direct message',
 'applesupport suffer hope find solution',
 'applesupport hi ’ concern latest ios slow happy solution please',
 'thank reach us always happy help send us direct message look together',
 'update phone suddenly everything take age load wtf update 

In [43]:
X=bow_model.toarray()

In [44]:
data["inbound"]=data["inbound"].map({True:'1',  False:'0'})

In [45]:
y=data["inbound"]
y.shape

(93,)

In [46]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state =42 )

In [47]:
print("The shape of respective train and test values : ")
print("X_train : " , X_train.shape)
print("X_test : " , X_test.shape)
print("y_train : " , y_train.shape)
print("y_test : " , y_test.shape)

The shape of respective train and test values : 
X_train :  (74, 459)
X_test :  (19, 459)
y_train :  (74,)
y_test :  (19,)


In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report


MLP CLASSIFIER

In [49]:
MLP = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000,
                    activation="relu",
                   alpha=0.001,
                   batch_size="auto",
                    early_stopping=False,
                    verbose=True,
                    learning_rate="adaptive")
MLP.fit(X_train,y_train)

y_pred1 = MLP.predict(X_test)



Iteration 1, loss = 0.70638258
Iteration 2, loss = 0.68118720
Iteration 3, loss = 0.65754998
Iteration 4, loss = 0.63500735
Iteration 5, loss = 0.61337655
Iteration 6, loss = 0.59244274
Iteration 7, loss = 0.57187569
Iteration 8, loss = 0.55150203
Iteration 9, loss = 0.53114238
Iteration 10, loss = 0.51073696
Iteration 11, loss = 0.49028509
Iteration 12, loss = 0.46983285
Iteration 13, loss = 0.44927436
Iteration 14, loss = 0.42860617
Iteration 15, loss = 0.40799357
Iteration 16, loss = 0.38739365
Iteration 17, loss = 0.36691032
Iteration 18, loss = 0.34656868
Iteration 19, loss = 0.32650571
Iteration 20, loss = 0.30675537
Iteration 21, loss = 0.28733126
Iteration 22, loss = 0.26838093
Iteration 23, loss = 0.24996483
Iteration 24, loss = 0.23212352
Iteration 25, loss = 0.21492385
Iteration 26, loss = 0.19845311
Iteration 27, loss = 0.18284179
Iteration 28, loss = 0.16804851
Iteration 29, loss = 0.15405271
Iteration 30, loss = 0.14089853
Iteration 31, loss = 0.12860279
Iteration 32, los

In [50]:

print("Accuracy score: {}". format(accuracy_score(y_true=y_test, y_pred=y_pred1)) )
Accuracy_Score_mlp_coun=(accuracy_score(y_true=y_test, y_pred=y_pred1))
print(Accuracy_Score_mlp_coun)
print("--"*30)
print(classification_report(y_pred1,y_test))


Accuracy score: 0.8947368421052632
0.8947368421052632
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.88      0.88         8
           1       0.91      0.91      0.91        11

    accuracy                           0.89        19
   macro avg       0.89      0.89      0.89        19
weighted avg       0.89      0.89      0.89        19



MultiNomial CLASSIFIER

In [51]:
MNC = MultinomialNB()
MNC.fit(X_train,y_train)
y_pred2 = MNC.predict(X_test)

print("Prediction results of MNC Classifier :: ")
print("----------------------------------------")
print("Accuracy score: {}". format(accuracy_score(y_test, y_pred2)) )
Accuracy_Score_ml_coun=(accuracy_score(y_true=y_test, y_pred=y_pred2))
print(Accuracy_Score_ml_coun)

print("--"*30)
print(classification_report(y_pred2,y_test))


Prediction results of MNC Classifier :: 
----------------------------------------
Accuracy score: 0.7894736842105263
0.7894736842105263
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.73      0.89      0.80         9

    accuracy                           0.79        19
   macro avg       0.80      0.79      0.79        19
weighted avg       0.81      0.79      0.79        19



Bernoulli CLASSIFIER

In [52]:
BC = BernoulliNB()
BC.fit(X_train,y_train)
y_pred3 = BC.predict(X_test)
print("Prediction results of MNC Classifier :: ")
print("----------------------------------------")
print("Accuracy score: {}". format(accuracy_score(y_test, y_pred3)) )
Accuracy_Score_BC_coun=(accuracy_score(y_true=y_test, y_pred=y_pred3))
print(Accuracy_Score_BC_coun)
print("--"*30)
print(classification_report(y_pred3,y_test))


Prediction results of MNC Classifier :: 
----------------------------------------
Accuracy score: 0.8421052631578947
0.8421052631578947
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.75      0.86      0.80         7
           1       0.91      0.83      0.87        12

    accuracy                           0.84        19
   macro avg       0.83      0.85      0.83        19
weighted avg       0.85      0.84      0.84        19



In [53]:
compare_df =[]
models = [("Multinomial NB", MNC), ("Bernoulli NB", BC),("MLP Classifier", MLP) ]
for model_name, model in models:
    predictions = model.predict(X_test)
    Accuracy_Score = accuracy_score(y_test,predictions)


    compare_df.append([model_name,Accuracy_Score])

compare_df = pd.DataFrame(compare_df, columns=["Model_Name","Accuracy_Score"])
compare_df

,Model_Name,Accuracy_Score
0,Multinomial NB,0.789474
1,Bernoulli NB,0.842105
2,MLP Classifier,0.894737


In [54]:

import plotly.express as px

In [55]:
fig = px.bar(compare_df,x="Model_Name",y="Accuracy_Score",title="Accuracy_Score Comparison",)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

TD-IDF

In [56]:
# bag of words model using Tfidf

vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(documents)
print(tfidf_model)

  (0, 304)	0.3780748993488346
  (0, 229)	0.3780748993488346
  (0, 294)	0.3780748993488346
  (0, 434)	0.3780748993488346
  (0, 125)	0.3780748993488346
  (0, 356)	0.29243673643702095
  (0, 72)	0.3780748993488346
  (0, 27)	0.238405071414758
  (1, 358)	0.3052457302012442
  (1, 91)	0.2797276422667727
  (1, 117)	0.3052457302012442
  (1, 1)	0.3052457302012442
  (1, 84)	0.2797276422667727
  (1, 509)	0.3052457302012442
  (1, 288)	0.2361041828090945
  (1, 270)	0.14885726389373802
  (1, 121)	0.160331734632603
  (1, 323)	0.19248072335141625
  (1, 473)	0.15726110864107937
  (1, 260)	0.2797276422667727
  (1, 269)	0.2797276422667727
  (1, 58)	0.3052457302012442
  (1, 356)	0.2361041828090945
  (2, 129)	0.37447611572895667
  (2, 498)	0.37447611572895667
  :	:
  (91, 202)	0.26429752257931627
  (91, 146)	0.26429752257931627
  (91, 219)	0.26429752257931627
  (91, 354)	0.26429752257931627
  (91, 101)	0.26429752257931627
  (91, 239)	0.26429752257931627
  (91, 148)	0.26429752257931627
  (91, 48)	0.2642975225

In [57]:
# print the full sparse matrix
print(tfidf_model.toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.30524573 0.         ... 0.         0.         0.30524573]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.27726823 ... 0.         0.         0.        ]]


In [58]:
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ages,ago,...,working,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.305246,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.305246
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [59]:
print(tfidf_model.shape)
print(vectorizer.get_feature_names_out())

(93, 510)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ages' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow'
 'almost' 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another'
 'answer' 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps'
 'armv' 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance'
 'attached' 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg'
 'behaviors' 'believes' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairways' 'broadside' 'broken' 'browser' 'browsers' 'bug'
 'business' 'buying' 'bye' 'cable' 'cache' 'cachecookies' 'call' 'called'
 'calling' 'calls' 'cares' 'carry' 'cause' 'caused' 'causing' 'challenged'
 'change' 'charge' 'chat' 'check' 'cheers' 'choice' 'chris' 'click'
 'close' 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'comes'
 'commonsense' 'companion' 'concern' 'congrats' 'constantly' 'contact'
 'cookies' 'correct' 'could' 'coupling' 'crash' 'crossed' 'cumbersome'
 'curiousw

In [60]:
# stem messages
stem_messages = [preprocess(message, stem=True) for message in  documents]

# bag of words model
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(stem_messages)
print(tfidf_model)


  (0, 271)	0.3854570571559503
  (0, 203)	0.3854570571559503
  (0, 261)	0.3854570571559503
  (0, 389)	0.3854570571559503
  (0, 110)	0.3854570571559503
  (0, 320)	0.29814675352806336
  (0, 64)	0.3303703888037585
  (0, 26)	0.24306008517587152
  (1, 321)	0.28197872663376644
  (1, 81)	0.28197872663376644
  (1, 104)	0.307702169206626
  (1, 1)	0.307702169206626
  (1, 76)	0.28197872663376644
  (1, 452)	0.307702169206626
  (1, 255)	0.2380042111030323
  (1, 239)	0.15005518004156396
  (1, 107)	0.15852665401346133
  (1, 289)	0.1940296955722981
  (1, 422)	0.15852665401346133
  (1, 230)	0.28197872663376644
  (1, 237)	0.28197872663376644
  (1, 55)	0.307702169206626
  (1, 320)	0.2380042111030323
  (2, 114)	0.3768990345608239
  (2, 442)	0.3768990345608239
  :	:
  (91, 178)	0.2713602124888724
  (91, 131)	0.2713602124888724
  (91, 193)	0.2713602124888724
  (91, 318)	0.2713602124888724
  (91, 91)	0.2713602124888724
  (91, 213)	0.2713602124888724
  (91, 46)	0.2713602124888724
  (91, 394)	0.1872088019534454

In [61]:
# look at the dataframe
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,addit,address,adnam,advertis,advis,affect,age,ago,agoura,...,work,wtf,xfiniti,yard,year,yep,youll,your,youv,zip
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.307702,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307702
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [62]:
print(vectorizer.get_feature_names_out())

['account' 'addit' 'address' 'adnam' 'advertis' 'advis' 'affect' 'age'
 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost' 'also' 'alway'
 'amanda' 'amazon' 'android' 'anker' 'anoth' 'answer' 'anyth' 'app' 'appl'
 'applesupport' 'appreci' 'armv' 'articl' 'ask' 'askspectrum' 'ass'
 'assess' 'assist' 'attach' 'avoid' 'away' 'ba' 'back' 'basket' 'batteri'
 'beatner' 'beg' 'behavior' 'believ' 'bit' 'blind' 'bluetooth' 'brilliant'
 'britishairway' 'broadsid' 'broken' 'browser' 'bug' 'busi' 'buy' 'bye'
 'cabl' 'cach' 'cachecooki' 'call' 'care' 'carri' 'caus' 'challeng'
 'chang' 'charg' 'chat' 'check' 'cheer' 'choic' 'chri' 'click' 'close'
 'coaxial' 'code' 'colleagu' 'come' 'commonsens' 'companion' 'concern'
 'congrat' 'constantli' 'contact' 'cooki' 'correct' 'could' 'coupl'
 'crash' 'cross' 'cumbersom' 'curiouswil' 'current' 'custom' 'dani' 'day'
 'dead' 'delay' 'delet' 'deliv' 'deliveri' 'depart' 'departur' 'detail'
 'devic' 'differ' 'direct' 'disconnect' 'disgrac' 'disregard' 'dis

In [63]:
# lemmatise messages
lemma_messages = [preprocess(message, stem=False) for message in documents]

# bag of words model
vectorizer = TfidfVectorizer()
tfidf_model = vectorizer.fit_transform(lemma_messages)

In [64]:
# look at the dataframe
pd.DataFrame(tfidf_model.toarray(), columns = vectorizer.get_feature_names_out())

,account,additional,address,adnams,advertisement,advise,affect,age,ago,agoura,...,work,wtf,xfinity,yard,year,yep,youll,youre,youve,zip
0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.307702,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307702
2,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
89,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
90,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
91,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [65]:
print(tfidf_model.shape)
print(vectorizer.get_feature_names_out())

(93, 459)
['account' 'additional' 'address' 'adnams' 'advertisement' 'advise'
 'affect' 'age' 'ago' 'agoura' 'air' 'aircraft' 'alcohol' 'allow' 'almost'
 'also' 'always' 'amanda' 'amazon' 'android' 'anker' 'another' 'answer'
 'anything' 'app' 'apple' 'applesupport' 'appreciate' 'apps' 'armv'
 'article' 'ask' 'askspectrum' 'ass' 'assessment' 'assistance' 'attach'
 'avoid' 'away' 'ba' 'back' 'basket' 'battery' 'beatner' 'beg' 'behaviors'
 'believe' 'bite' 'blind' 'bluetooth' 'break' 'brilliant' 'britishairways'
 'broadside' 'browser' 'browsers' 'bug' 'business' 'buy' 'bye' 'cable'
 'cache' 'cachecookies' 'call' 'care' 'carry' 'cause' 'challenge' 'change'
 'charge' 'chat' 'check' 'cheer' 'choice' 'chris' 'click' 'close'
 'coaxial' 'code' 'colleague' 'colleagues' 'come' 'commonsense'
 'companion' 'concern' 'congrats' 'constantly' 'contact' 'cookies'
 'correct' 'could' 'couple' 'crash' 'cross' 'cumbersome' 'curiouswill'
 'currently' 'customer' 'dani' 'day' 'days' 'dead' 'delay' 'delete'
 'd

In [66]:
X=tfidf_model.toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.30770217, 0.        , ..., 0.        , 0.        ,
        0.30770217],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.28233002, ..., 0.        , 0.        ,
        0.        ]])

In [67]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state =42 )

In [68]:
print("The shape of respective train and test values : ")
print("X_train : " , X_train.shape)
print("X_test : " , X_test.shape)
print("y_train : " , y_train.shape)
print("y_test : " , y_test.shape)

The shape of respective train and test values : 
X_train :  (74, 459)
X_test :  (19, 459)
y_train :  (74,)
y_test :  (19,)


MLP CLASSIFIER

In [69]:
MLP = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000,
                    activation="relu",
                   alpha=0.001,
                   batch_size="auto",
                    early_stopping=False,
                    verbose=True,
                    learning_rate="adaptive")
MLP.fit(X_train,y_train)

y_pred1 = MLP.predict(X_test)



Iteration 1, loss = 0.70714937
Iteration 2, loss = 0.69775064
Iteration 3, loss = 0.68866996
Iteration 4, loss = 0.67985095
Iteration 5, loss = 0.67111873
Iteration 6, loss = 0.66247985
Iteration 7, loss = 0.65381315
Iteration 8, loss = 0.64489769
Iteration 9, loss = 0.63564798
Iteration 10, loss = 0.62606232
Iteration 11, loss = 0.61610693
Iteration 12, loss = 0.60577026
Iteration 13, loss = 0.59505670
Iteration 14, loss = 0.58393326
Iteration 15, loss = 0.57227235
Iteration 16, loss = 0.56019191
Iteration 17, loss = 0.54763467
Iteration 18, loss = 0.53461263
Iteration 19, loss = 0.52114289
Iteration 20, loss = 0.50731599
Iteration 21, loss = 0.49308683
Iteration 22, loss = 0.47852334
Iteration 23, loss = 0.46362314
Iteration 24, loss = 0.44840568
Iteration 25, loss = 0.43292829
Iteration 26, loss = 0.41723006
Iteration 27, loss = 0.40138679
Iteration 28, loss = 0.38544437
Iteration 29, loss = 0.36945034
Iteration 30, loss = 0.35346868
Iteration 31, loss = 0.33755343
Iteration 32, los

In [70]:

print("Accuracy score: {}". format(accuracy_score(y_true=y_test, y_pred=y_pred1)) )
Accuracy_Score_mlp_tf=(accuracy_score(y_true=y_test, y_pred=y_pred1))
print(Accuracy_Score_mlp_tf)

print("--"*30)
print(classification_report(y_pred1,y_test))


Accuracy score: 0.7368421052631579
0.7368421052631579
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.64      0.74        11
           1       0.64      0.88      0.74         8

    accuracy                           0.74        19
   macro avg       0.76      0.76      0.74        19
weighted avg       0.77      0.74      0.74        19



MultiNomial CLASSIFIER

In [71]:
MNC = MultinomialNB()
MNC.fit(X_train,y_train)
y_pred2 = MNC.predict(X_test)

print("Prediction results of MNC Classifier :: ")
print("----------------------------------------")
print("Accuracy score: {}". format(accuracy_score(y_test, y_pred2)) )
Accuracy_Score_ml_tf=(accuracy_score(y_true=y_test, y_pred=y_pred2))
print(Accuracy_Score_ml_tf)

print("--"*30)
print(classification_report(y_pred2,y_test))


Prediction results of MNC Classifier :: 
----------------------------------------
Accuracy score: 0.7894736842105263
0.7894736842105263
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        10
           1       0.73      0.89      0.80         9

    accuracy                           0.79        19
   macro avg       0.80      0.79      0.79        19
weighted avg       0.81      0.79      0.79        19



Bernoulli CLASSIFIER

In [72]:
BC = BernoulliNB()
BC.fit(X_train,y_train)
y_pred3 = BC.predict(X_test)
print("Prediction results of MNC Classifier :: ")
print("----------------------------------------")
print("Accuracy score: {}". format(accuracy_score(y_test, y_pred3)) )
Accuracy_Score_BC_tf=(accuracy_score(y_true=y_test, y_pred=y_pred3))
print(Accuracy_Score_BC_tf)
print("--"*30)
print(classification_report(y_pred3,y_test))


Prediction results of MNC Classifier :: 
----------------------------------------
Accuracy score: 0.8421052631578947
0.8421052631578947
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.75      0.86      0.80         7
           1       0.91      0.83      0.87        12

    accuracy                           0.84        19
   macro avg       0.83      0.85      0.83        19
weighted avg       0.85      0.84      0.84        19



In [73]:
compare_df =[]
models = [("Multinomial NB", MNC), ("Bernoulli NB", BC),("MLP Classifier", MLP) ]
for model_name, model in models:
    predictions = model.predict(X_test)
    Accuracy_Score = accuracy_score(y_test,predictions)


    compare_df.append([model_name,Accuracy_Score])

compare_df = pd.DataFrame(compare_df, columns=["Model_Name","Accuracy_Score"])
compare_df

,Model_Name,Accuracy_Score
0,Multinomial NB,0.789474
1,Bernoulli NB,0.842105
2,MLP Classifier,0.736842


In [74]:
fig = px.bar(compare_df,x="Model_Name",y="Accuracy_Score",title="Accuracy_Score Comparison",)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

 N-grams

In [75]:

bigram=CountVectorizer(ngram_range=(2,2),stop_words='english')

bigramvocab2=bigram.fit_transform(lemma_messages).toarray()

pd.DataFrame(bigramvocab2, columns = bigram.get_feature_names_out())



,account ass,account information,account prior,additional concern,address account,address email,adnams broadside,affect playback,age load,age paige,...,xfinity talkin,yard ive,yep ive,youll basket,youll receive,youre experience,youre use,youre welcome,youve try,zip code
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:

bigram=CountVectorizer(ngram_range=(3,3),stop_words='english')

bigramvocab3=bigram.fit_transform(lemma_messages).toarray()

pd.DataFrame(bigramvocab3, columns = bigram.get_feature_names_out())

,account information look,account prior delete,additional concern reply,address account ass,address email id,address email include,adnams broadside blind,affect playback log,age load wtf,age protect make,...,yard ive time,yep ive try,youll basket thank,youll receive congrats,youre experience direct,youre use thatd,youre use use,youre welcome theres,youve try experience,zip code additional
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
bigram.vocabulary_

{'applesupport cause reply': 24,
 'cause reply disregard': 77,
 'reply disregard tap': 422,
 'disregard tap notification': 139,
 'tap notification keyboard': 501,
 'notification keyboard open': 356,
 'business mean lot': 69,
 'mean lot direct': 323,
 'lot direct message': 314,
 'direct message zip': 136,
 'message zip code': 333,
 'zip code additional': 608,
 'code additional concern': 86,
 'additional concern reply': 2,
 'concern reply request': 95,
 'really hope change': 413,
 'hope change im': 217,
 'change im sure': 79,
 'im sure wont': 230,
 'sure wont dont': 495,
 'livechat online moment': 300,
 'online moment contact': 365,
 'moment contact option': 345,
 'contact option leave': 98,
 'option leave message': 369,
 'leave message request': 280,
 'virgintrains attach error': 573,
 'attach error message': 46,
 'error message ive': 151,
 'message ive try': 329,
 'ive try leave': 261,
 'try leave voicemail': 543,
 'leave voicemail time': 281,
 'voicemail time past': 577,
 'time past w

In [78]:

bigram=CountVectorizer(ngram_range=(4,4),stop_words='english')

bigramvocab4=bigram.fit_transform(lemma_messages).toarray()
_
pd.DataFrame(bigramvocab4, columns = bigram.get_feature_names_out())


,account information look matter,account prior delete cache,additional concern reply request,address email id happy,address email include browser,adnams broadside blind jobspec,affect playback log restart,age load wtf update,age protect make underage,ago ive day telemarketer,...,yard ive time weeks,yep ive try laptop,youll basket thank mike,youll receive congrats email,youre experience direct message,youre use thatd great,youre use use specific,youre welcome theres help,youve try experience behaviors,zip code additional concern
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
bigram.vocabulary_

{'applesupport cause reply disregard': 23,
 'cause reply disregard tap': 70,
 'reply disregard tap notification': 362,
 'disregard tap notification keyboard': 118,
 'tap notification keyboard open': 433,
 'business mean lot direct': 63,
 'mean lot direct message': 275,
 'lot direct message zip': 268,
 'direct message zip code': 116,
 'message zip code additional': 284,
 'zip code additional concern': 530,
 'code additional concern reply': 77,
 'additional concern reply request': 2,
 'really hope change im': 354,
 'hope change im sure': 184,
 'change im sure wont': 72,
 'im sure wont dont': 196,
 'livechat online moment contact': 256,
 'online moment contact option': 312,
 'moment contact option leave': 295,
 'contact option leave message': 87,
 'option leave message request': 316,
 'virgintrains attach error message': 495,
 'attach error message ive': 45,
 'error message ive try': 127,
 'message ive try leave': 281,
 'ive try leave voicemail': 225,
 'try leave voicemail time': 470,
 'l

In [80]:
len(bigram.vocabulary_)


531

OneHotEncoder

In [81]:
values = np.array(lemma_messages)
print(values)

['applesupport cause reply disregard tap notification keyboard open'
 'business mean lot us please direct message name zip code additional detail concern reply request'
 'really hope change im sure wont dont'
 'livechat online moment contact option leave message request call back'
 'virgintrains see attach error message ive try leave voicemail several time past week'
 'try another device miriam message'
 'virgintrains yep ive try laptop several time past week today ive try different browsers'
 'work ok miriam link help message'
 'virgintrains still havent hear number im direct phone dead end live chat doesnt work someone call'
 'thats miriam team send email shortly happy'
 'help version ios find settings general reply direct message'
 'applesupport suffer hope find solution'
 'applesupport hi ’ concern latest ios slow happy solution please'
 'thank reach us always happy help send us direct message look together'
 'update phone suddenly everything take age load wtf update sux hate fix b

In [82]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = values.reshape(len(values), 1)
integer_encoded

array([['applesupport cause reply disregard tap notification keyboard open'],
       ['business mean lot us please direct message name zip code additional detail concern reply request'],
       ['really hope change im sure wont dont'],
       ['livechat online moment contact option leave message request call back'],
       ['virgintrains see attach error message ive try leave voicemail several time past week'],
       ['try another device miriam message'],
       ['virgintrains yep ive try laptop several time past week today ive try different browsers'],
       ['work ok miriam link help message'],
       ['virgintrains still havent hear number im direct phone dead end live chat doesnt work someone call'],
       ['thats miriam team send email shortly happy'],
       ['help version ios find settings general reply direct message'],
       ['applesupport suffer hope find solution'],
       ['applesupport hi ’ concern latest ios slow happy solution please'],
       ['thank reach us always

In [83]:
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [84]:
(onehot_encoded).shape

(93, 91)

In [121]:
print("Accuracy_Score for countvecter MLP CLASSIFIER  : {0:.3}".format(Accuracy_Score_mlp_coun))
print("Accuracy_Score for countvecter MultiNomial CLASSIFIER : {0:.3}".format(Accuracy_Score_ml_coun))
print("Accuracy_Score for countvecter Bernoulli CLASSIFIER : {0:.3}".format(Accuracy_Score_BC_coun))
print("Accuracy_Score for TF-IDF MLP CLASSIFIER  : {0:.3}".format(Accuracy_Score_mlp_tf))
print("Accuracy_Score for TF-IDF MultiNomial CLASSIFIER : {0:.3}".format(Accuracy_Score_ml_tf))
print("Accuracy_Score for TF-IDF Bernoulli CLASSIFIER : {0:.3}".format(Accuracy_Score_BC_tf))





Accuracy_Score for countvecter MLP CLASSIFIER  : 0.895
Accuracy_Score for countvecter MultiNomial CLASSIFIER : 0.789
Accuracy_Score for countvecter Bernoulli CLASSIFIER : 0.842
Accuracy_Score for countvecter MLP CLASSIFIER  : 0.737
Accuracy_Score for countvecter MultiNomial CLASSIFIER : 0.789
Accuracy_Score for countvecter Bernoulli CLASSIFIER : 0.842


Summary:

*   Analysis All result in the above method two method like countvector and TF-IDF
*   Countvector perform better for MLP CLASSIFIER with 89%
*   TFIDF perform better Bernoulli CLASSIFIER with 84%








Word2vec Implementation

In [86]:
pip install gensim

In [87]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [88]:
words=[]
for sent in lemma_messages:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [89]:
words

[['applesupport',
  'cause',
  'reply',
  'disregard',
  'tap',
  'notification',
  'keyboard',
  'open'],
 ['business',
  'mean',
  'lot',
  'us',
  'please',
  'direct',
  'message',
  'name',
  'zip',
  'code',
  'additional',
  'detail',
  'concern',
  'reply',
  'request'],
 ['really', 'hope', 'change', 'im', 'sure', 'wont', 'dont'],
 ['livechat',
  'online',
  'moment',
  'contact',
  'option',
  'leave',
  'message',
  'request',
  'call',
  'back'],
 ['virgintrains',
  'see',
  'attach',
  'error',
  'message',
  'ive',
  'try',
  'leave',
  'voicemail',
  'several',
  'time',
  'past',
  'week'],
 ['try', 'another', 'device', 'miriam', 'message'],
 ['virgintrains',
  'yep',
  'ive',
  'try',
  'laptop',
  'several',
  'time',
  'past',
  'week',
  'today',
  'ive',
  'try',
  'different',
  'browsers'],
 ['work', 'ok', 'miriam', 'link', 'help', 'message'],
 ['virgintrains',
  'still',
  'havent',
  'hear',
  'number',
  'im',
  'direct',
  'phone',
  'dead',
  'end',
  'live',

In [90]:
import gensim

In [91]:
model = gensim.models.Word2Vec(words, window=5, min_count=2)

In [92]:
model.build_vocab(words)

In [93]:
model.wv.index_to_key

['us',
 'message',
 'help',
 'direct',
 'thank',
 'get',
 'applesupport',
 'please',
 'use',
 'update',
 'try',
 'phone',
 'hi',
 'ive',
 'let',
 'tesco',
 'look',
 'version',
 'send',
 'spotifycares',
 'time',
 'device',
 'ios',
 'happy',
 'call',
 'issue',
 'know',
 'back',
 'wed',
 'happen',
 'apps',
 'battery',
 'reply',
 'assessment',
 'year',
 'account',
 'log',
 'sorry',
 'name',
 'well',
 'experience',
 'still',
 'like',
 'run',
 'go',
 'spotify',
 'hear',
 'full',
 'email',
 'app',
 'youre',
 'store',
 'take',
 'speaker',
 'delay',
 'work',
 'find',
 'virgintrains',
 'cause',
 'restart',
 'make',
 'number',
 'see',
 'could',
 'fix',
 'age',
 'everything',
 'miriam',
 'today',
 'always',
 'reach',
 'latest',
 'love',
 'im',
 'hope',
 'together',
 'up',
 'id',
 'music',
 'address',
 'iphone',
 'life',
 'weeks',
 'one',
 'tell',
 'service',
 'thomas',
 'start',
 'delete',
 'new',
 'point',
 'problem',
 'youll',
 'immediately',
 'link',
 'answer',
 'week',
 'past',
 'several',
 's

In [94]:
model.epochs

5

In [95]:
model.corpus_total_words

973

In [96]:
model.corpus_count

93

In [97]:
model.train(words,total_examples=model.corpus_count, epochs=5)

(1887, 4865)

In [98]:
model.wv.most_similar(["applesupport"])

[('great', 0.3080560266971588),
 ('since', 0.24761123955249786),
 ('wont', 0.24204114079475403),
 ('name', 0.2312961220741272),
 ('app', 0.22327019274234772),
 ('could', 0.21973802149295807),
 ('depart', 0.21101205050945282),
 ('time', 0.20552067458629608),
 ('thank', 0.1907845288515091),
 ('thats', 0.17735600471496582)]

In [99]:
model.wv["applesupport"]

array([-0.00079046,  0.0005276 ,  0.00507982,  0.00889068, -0.00914201,
       -0.00780411,  0.00671217,  0.00990665, -0.00563913, -0.0038691 ,
        0.00734255, -0.00204589, -0.00447521,  0.00697113, -0.00459973,
       -0.00176349,  0.00305803,  0.00030962, -0.00874035, -0.01039464,
        0.00768561,  0.00530567,  0.0065575 ,  0.00048939,  0.00612329,
       -0.00333455, -0.00103606,  0.00522987, -0.00783895, -0.00386838,
       -0.00701115, -0.00092901,  0.00963518, -0.00780277, -0.00254891,
       -0.00096124,  0.00836398, -0.00610516, -0.00032688, -0.00534071,
       -0.00941493,  0.00450967, -0.00886271, -0.00416673,  0.00043911,
       -0.00042285, -0.00774128,  0.0094192 ,  0.00525541,  0.00936454,
       -0.00785898,  0.00394656, -0.00407849,  0.00088085,  0.00838749,
       -0.00424061,  0.00455649, -0.00654335, -0.00402848,  0.00971602,
       -0.00143331,  0.00045495, -0.00374538, -0.00776624, -0.00168865,
        0.00310135, -0.00058935,  0.00623437, -0.00316531,  0.00

In [100]:
vectors=model.wv.get_normed_vectors()

In [101]:
model.wv.get_normed_vectors()

array([[-0.13796264,  0.03934667, -0.01353902, ..., -0.14178671,
         0.05210736, -0.10771482],
       [-0.17067382,  0.16000171,  0.0704053 , ...,  0.11591944,
         0.10723778, -0.09946217],
       [ 0.08669809,  0.17668433, -0.12080074, ...,  0.14379802,
        -0.0095096 , -0.01683003],
       ...,
       [-0.12402003, -0.0982659 ,  0.05024147, ...,  0.03387966,
        -0.13522066, -0.14495365],
       [ 0.1518633 ,  0.02434363, -0.07686705, ..., -0.04461791,
         0.09525181,  0.09966429],
       [ 0.16296151,  0.13334498,  0.12178508, ..., -0.07058505,
         0.06845352,  0.02249745]], dtype=float32)

In [102]:
model.wv.get_normed_vectors().shape

(171, 100)

In [103]:
y=model.wv.index_to_key

In [104]:
X=model.wv.get_normed_vectors()

In [105]:
X

array([[-0.13796264,  0.03934667, -0.01353902, ..., -0.14178671,
         0.05210736, -0.10771482],
       [-0.17067382,  0.16000171,  0.0704053 , ...,  0.11591944,
         0.10723778, -0.09946217],
       [ 0.08669809,  0.17668433, -0.12080074, ...,  0.14379802,
        -0.0095096 , -0.01683003],
       ...,
       [-0.12402003, -0.0982659 ,  0.05024147, ...,  0.03387966,
        -0.13522066, -0.14495365],
       [ 0.1518633 ,  0.02434363, -0.07686705, ..., -0.04461791,
         0.09525181,  0.09966429],
       [ 0.16296151,  0.13334498,  0.12178508, ..., -0.07058505,
         0.06845352,  0.02249745]], dtype=float32)

RNN
